# Введение

### Задача - найти идентичные товары по описанию; решим ее посредством обучения эмбеддингов.

# Библиотеки и установки

In [ ]:
#!pip install --upgrade nltk gensim bokeh
#!pip install pytorch-metric-learning
#!pip install record-keeper
#!pip install faiss-cpu

In [1]:
import glob
import os
import random
import string
import csv
import time
import logging
import record_keeper
import unicodedata

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from cycler import cycler

import pytorch_metric_learning
import pytorch_metric_learning.utils.logging_presets as logging_presets
from pytorch_metric_learning import losses, miners, samplers, testers, trainers
from pytorch_metric_learning.utils import common_functions
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning.distances import SNRDistance
from pytorch_metric_learning.utils.inference import CustomKNN

logging.getLogger().setLevel(logging.INFO)
logging.info("VERSION %s" % pytorch_metric_learning.__version__)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%matplotlib inline

INFO:root:VERSION 1.3.0


# Загрузка, первичный анализ данных и постановка вопроса

In [4]:
df = pd.read_csv("train_data.csv")
df.head()

,posting_id,title,label_group
0,train_129225211,Paper Bag Victoria Secret,249114794
1,train_2278313361,PAPER BAG VICTORIA SECRET,249114794
2,train_2288590299,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_3803689425,Maling Ham Pork Luncheon Meat TTS 397gr,2395904891
4,train_2406599165,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188


Уникальным идентификатором товара является posting_id. Важно отметить, что все posting_id разные; мы будем действовать исходя из этого предположения касательно всего распределения - потенциальные повторы posting_id на инференсе (если они случатся) должны удаляться в контексте препроцессинга. <br>
<br>
Имеет смысл заострить внимание, что дубликаты в колонке title удалять нельзя, поскольку эти объекты (как и все остальные) имеют разный posting_id - в противном случае мы на ровном месте занижаем себе метрику, убирая самые легкие для модели случаи. <br>
<br>
Отметим также, что в среднем одному классу принадлежит 3-4 объекта.

In [36]:
print(df.shape[0], df['posting_id'].unique().shape[0], 
      df.label_group.unique().shape[0])

20952 20952 6608


In [37]:
tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
df['matches'] = df['label_group'].map(tmp)
df['matches'] = df['matches'].apply(lambda x: ' '.join(x))

нашей метрикой будет построчный f1

In [38]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)]) 
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

в качестве бейзлайна сопоставим себя в группу только с самим собой и каким нибудь еще одним случайным товаром

In [39]:
df['prediction'] = df['posting_id'] + ' ' + 'train_129225211'

In [40]:
f1_score(df['matches'], df['prediction']).mean()

0.35675992543848534

цель улучшить эту метрику, пользоваться любыми методами

на проде данных много и методы должны адаптироваться на большие данные

# Препроцессинг

### Label encoding

Подобное переобозначение понадобится в дальнейшем, поскольку будет использоваться дизъюнктивный даталоадер

In [41]:
#хэш таблица с использованием label encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

original_labels = list(set(df['label_group']))
le.fit(original_labels)
new_labels = le.transform(original_labels)
hash_table = dict(zip(original_labels, new_labels))

In [42]:
df['label_group'] = df['label_group'].apply(lambda x: hash_table[x])
df

,posting_id,title,label_group,matches,prediction
0,train_129225211,Paper Bag Victoria Secret,389,train_129225211 train_2278313361,train_129225211 train_129225211
1,train_2278313361,PAPER BAG VICTORIA SECRET,389,train_129225211 train_2278313361,train_2278313361 train_129225211
2,train_2288590299,Maling TTS Canned Pork Luncheon Meat 397 gr,3672,train_2288590299 train_3803689425,train_2288590299 train_129225211
3,train_3803689425,Maling Ham Pork Luncheon Meat TTS 397gr,3672,train_2288590299 train_3803689425,train_3803689425 train_129225211
4,train_2406599165,Daster Batik Lengan pendek - Motif Acak / Camp...,6300,train_2406599165 train_3342059966,train_2406599165 train_129225211
...,...,...,...,...,...
20947,train_1001474240,Sam A20 A30 A30S A50 A50S A51 A71 A70 A70s M10...,5552,train_2944131255 train_1001474240,train_1001474240 train_129225211
20948,train_2244662893,LAMPU HURUF A-Z DAN ANGKA 0-9 \xe2\x9d\xa4\xef...,1148,train_2244662893 train_3281898016,train_2244662893 train_129225211
20949,train_3281898016,LAMPU HURUF A-Z DAN ANGKA 0-9 TINGGI 16 CM,1148,train_2244662893 train_3281898016,train_3281898016 train_129225211
20950,train_4221982820,Sprei Lady Rose 180x200 King terlaris Keroppi,75,train_4221982820 train_4063409014,train_4221982820 train_129225211


### Словарь символов

Нужно собрать словарь всех символов, которые могут встречаться в title; необходимо сделать это в начале, до основных преобразований соответствующего столбца. 

In [43]:
all_chars = sorted(list(set(''.join(list(df['title'])).lower())))
n_chars = len(all_chars)
print('Total number of characters:', n_chars)
print('Characters: ', *all_chars, sep='')

#служебные словари
char2idx = {key: value for value, key in enumerate(all_chars)}
idx2char = {key: value for key, value in enumerate(all_chars)}

def chars2indices(chars):
    indices = []
    for char in chars:
        idx = char2idx[char]
        indices.append(idx)
    return indices

Total number of characters: 69
Characters:  !"#$%&'()*+,-./0123456789:;<=>?@[\]^_`abcdefghijklmnopqrstuvwxyz{|}~


### Токенизация

Стандартный этап предобработки текстовых данных; использован токенизатор ntlk. Потенциально можно рассмотреть другие варианты токенизаций (напр., по n-грамам), и токенизаторов (например, YouTokenToMe от VK, в котором присутствует BPE).

In [10]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

df['title'] = df['title'].apply(lambda x: tokenizer.tokenize(x.lower()))
df

,posting_id,title,label_group,matches,prediction
0,train_129225211,"[paper, bag, victoria, secret]",389,train_129225211 train_2278313361,train_129225211 train_129225211
1,train_2278313361,"[paper, bag, victoria, secret]",389,train_129225211 train_2278313361,train_2278313361 train_129225211
2,train_2288590299,"[maling, tts, canned, pork, luncheon, meat, 39...",3672,train_2288590299 train_3803689425,train_2288590299 train_129225211
3,train_3803689425,"[maling, ham, pork, luncheon, meat, tts, 397gr]",3672,train_2288590299 train_3803689425,train_3803689425 train_129225211
4,train_2406599165,"[daster, batik, lengan, pendek, -, motif, acak...",6300,train_2406599165 train_3342059966,train_2406599165 train_129225211
...,...,...,...,...,...
20947,train_1001474240,"[sam, a20, a30, a30s, a50, a50s, a51, a71, a70...",5552,train_2944131255 train_1001474240,train_1001474240 train_129225211
20948,train_2244662893,"[lampu, huruf, a, -, z, dan, angka, 0, -, 9, \...",1148,train_2244662893 train_3281898016,train_2244662893 train_129225211
20949,train_3281898016,"[lampu, huruf, a, -, z, dan, angka, 0, -, 9, t...",1148,train_2244662893 train_3281898016,train_3281898016 train_129225211
20950,train_4221982820,"[sprei, lady, rose, 180x200, king, terlaris, k...",75,train_4221982820 train_4063409014,train_4221982820 train_129225211


### Загрузка предобученных эмбеддингов

В качестве предобученных эмбеддингов были выбраны twitter-25 из библиотеки gensim. Легкие, менее академичные, чем wiki и, если не ошибаюсь, был задействован меньший window size, что является плюсом с учетом меньшей связности подобных текстов. Возможно, очень интересным вариантом могло бы быть использование специальных эмбеддингов для ecommerce, например, как описано в статье "Query2Prod2Vec:
Grounded Word Embeddings for eCommerce" (https://arxiv.org/abs/2104.02061). С другой стороны, большой плюс эмбеддингов twitter в многоязычности, где все языки находятся в одном семантическом пространстве. Поскольку мы имеем дело с данными индонезийского ecommerce (смешанный индонезийско-английский текст), это очень весомое преимущество.

In [11]:
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-twitter-25')

INFO:gensim.models.keyedvectors:loading projection weights from /Users/andreyegorov/gensim-data/glove-twitter-25/glove-twitter-25.gz
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (1193514, 25) matrix of type float32 from /Users/andreyegorov/gensim-data/glove-twitter-25/glove-twitter-25.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-17T18:46:43.059618', 'gensim': '4.1.2', 'python': '3.8.11 (default, Aug  6 2021, 08:56:27) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


In [12]:
#семантическое пространство векторов покрывает индонезийский язык
print(df['title'][5][3])
glove_vectors.most_similar(df['title'][5][3])

jepang


[('inggris', 0.8793800473213196),
 ('eropa', 0.8693690896034241),
 ('indonesia', 0.8535388708114624),
 ('jawa', 0.8527381420135498),
 ('budaya', 0.8480342626571655),
 ('belanda', 0.844948947429657),
 ('perancis', 0.8439698815345764),
 ('versi', 0.8357038497924805),
 ('jerman', 0.8328619003295898),
 ('utama', 0.8298693895339966)]

### Каскадный принцип разделения на векторизуемые слова и символы

В связи со спецификой текста, кажется неверным полностью избавляться от невекторизируемых слов (для которых отсутствует предобученный эмбеддинг), заменяя их, допустим, на токен UNK. В таких словах может содержаться важная информация о технических характеристиках товара, указывающая на сходство (например, '10ml'). На мой взгляд, логично организовать векторизацию по каскадному принципу - отделить невекторизуемые слова, и векторизовать их уже на уровне символов, соответственно, в совокупности с обучаемыми с нуля эмбеддингами. Каждая из двух частей будет обрабатываться своим рекуррентным модулем в составе общей сети. По сути, мы отказываемся от токена UNK.  Кажется также, что не векторизуемые слова перед посимвольной векторизацией лучше отсортировать, чтобы зафиксировать некое примерное отношение порядка, скажем, тех же технических характеристик.

In [13]:
def split_vectorized(array):
    '''Разбиваем массив на векторизуемые данными эмбеддингами слова, и все остальные.
    Не векторизуемые упорядочиваем'''
    master_array = []
    word_level = []
    char_level = []
    for item in array:
        try:
            word_level.append(glove_vectors.key_to_index[item])
        except:
            char_level.append(item)
    master_array.append(word_level)
    master_array.append(sorted(char_level))
    
    return master_array

In [14]:
#разбиваем на часть, которая векторизуется по словам, и ту, которая нет
df['title'] = df['title'].apply(split_vectorized)
df_temp = pd.DataFrame(df["title"].to_list(), columns=['word_level', 'char_level'])
df = pd.concat([df[['posting_id']], df_temp, df[['label_group', 'matches']]], axis=1)
df

,posting_id,word_level,char_level,label_group,matches
0,train_129225211,"[2263, 2417, 3170, 2073]",[],389,train_129225211 train_2278313361
1,train_2278313361,"[2263, 2417, 3170, 2073]",[],389,train_129225211 train_2278313361
2,train_2288590299,"[23905, 22807, 62077, 12894, 80866, 7050, 3413]",[397],3672,train_2288590299 train_3803689425
3,train_3803689425,"[23905, 5442, 12894, 80866, 7050, 22807]",[397gr],3672,train_2288590299 train_3803689425
4,train_2406599165,"[131370, 18071, 46352, 11005, 28, 32674, 37431...","[00, alhadi, dpt001]",6300,train_2406599165 train_3342059966
...,...,...,...,...,...
20947,train_1001474240,"[2630, 28, 105361, 4088, 369, 234, 986]","[a20, a30, a30s, a31, a50, a50s, a51, a70, a70...",5552,train_2944131255 train_1001474240
20948,train_2244662893,"[8227, 14402, 11, 28, 1016, 233, 18988, 28, 37...","[0, 0, 16, 9, 9, x8f, x9d, xa4, xb8, xe2, xef]",1148,train_2244662893 train_3281898016
20949,train_3281898016,"[8227, 14402, 11, 28, 1016, 233, 18988, 28, 44...","[0, 16, 9]",1148,train_2244662893 train_3281898016
20950,train_4221982820,"[161903, 1404, 3902, 1696, 162980, 426544]",[180x200],75,train_4221982820 train_4063409014


In [15]:
#преобразуем char level
df['char_level'] = df['char_level'].apply(lambda x: ' '.join(x))
df['char_level'] = df['char_level'].apply(chars2indices)
#заплатка с empty list
df['char_level'] = df['char_level'].apply(lambda x: [0] if len(x) == 0 else x)
df['word_level'] = df['word_level'].apply(lambda x: [0] if len(x) == 0 else x)
df

,posting_id,word_level,char_level,label_group,matches
0,train_129225211,"[2263, 2417, 3170, 2073]",[0],389,train_129225211 train_2278313361
1,train_2278313361,"[2263, 2417, 3170, 2073]",[0],389,train_129225211 train_2278313361
2,train_2288590299,"[23905, 22807, 62077, 12894, 80866, 7050, 3413]","[19, 25, 23]",3672,train_2288590299 train_3803689425
3,train_3803689425,"[23905, 5442, 12894, 80866, 7050, 22807]","[19, 25, 23, 45, 56]",3672,train_2288590299 train_3803689425
4,train_2406599165,"[131370, 18071, 46352, 11005, 28, 32674, 37431...","[16, 16, 0, 39, 50, 46, 39, 42, 47, 0, 42, 54,...",6300,train_2406599165 train_3342059966
...,...,...,...,...,...
20947,train_1001474240,"[2630, 28, 105361, 4088, 369, 234, 986]","[39, 18, 16, 0, 39, 19, 16, 0, 39, 19, 16, 57,...",5552,train_2944131255 train_1001474240
20948,train_2244662893,"[8227, 14402, 11, 28, 1016, 233, 18988, 28, 37...","[16, 0, 16, 0, 17, 22, 0, 25, 0, 25, 0, 62, 24...",1148,train_2244662893 train_3281898016
20949,train_3281898016,"[8227, 14402, 11, 28, 1016, 233, 18988, 28, 44...","[16, 0, 17, 22, 0, 25]",1148,train_2244662893 train_3281898016
20950,train_4221982820,"[161903, 1404, 3902, 1696, 162980, 426544]","[17, 24, 16, 62, 18, 16, 16]",75,train_4221982820 train_4063409014


### Начало сборки в датасеты / даталоадеры

В начале прописан набор служебных упаковщиков/распаковщиков тензоров с кодированием разбиения; написал их, чтобы оптимальным образом состыковаться с интерфейсом классов библиотеки PyTorch Metric Learning (дальше PML). <br>
<br>
Даталоадер - на основе ноутбука: <br>
https://github.com/KevinMusgrave/pytorch-metric-learning/blob/master/examples/notebooks/MetricLossOnly.ipynb <br>
Его особенность в том, что он дизъюнктивный, то есть на тесте модель не видит тех классов, которые видела на трейне - это необходимо для специфики задач metric learning, чтобы подтолкнуть модель фокусироваться на выделении отношений между классами объектов, в противовес характеристикам конкретного класса (или классов).

In [16]:
def pack(tensor1, tensor2):
    '''input: two non-empty 1d tensors
    output: 1d tensor with splitting pointer at index [0]'''
    pointer = torch.tensor([tensor1.shape[0]])
    return torch.cat([pointer, tensor1, tensor2])

def unpack(packed):
    '''input: 1d tensor with splitting pointer at index [0]
    output: two 1d tensors split at pointer value'''
    pointer = int(packed[0])
    data = packed[1:]
    return data[:pointer], data[pointer:]

def batch_pack(tensor1, tensor2):
    '''input: two non-empty 2d tensors with batch size at dim 0
    output: 2d tensor with splitting pointer'''
    assert tensor1.shape[0] == tensor2.shape[0]
    bs = tensor1.shape[0]
    pointer = torch.tensor([tensor1.shape[1]]).expand(bs, -1)
    return torch.cat([pointer, tensor1, tensor2], dim=1)

def batch_unpack(packed):
    '''input: 2d tensor with splitting pointer
    output: two non-empty 2d tensors split at pointer value'''
    pointer = torch.unique(packed[:, 0])
    assert pointer.shape[0] == 1
    pointer = int(pointer[0])
    data = packed[:, 1:]
    return data[:, :pointer], data[:, pointer:]

In [17]:
class EntriesDisjointDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_flag):
        threshold = int(np.percentile(df['label_group'], 80))
        rule = (lambda x: x < threshold) if train_flag else (lambda x: x >= threshold)
        filtered_idx = [int(i) for i, x in enumerate(df['label_group']) if rule(x)]
        self.word_level = df['word_level'].iloc[filtered_idx].reset_index(drop=True)
        self.char_level = df['char_level'].iloc[filtered_idx].reset_index(drop=True)
        self.labels = df['label_group'].iloc[filtered_idx].reset_index(drop=True)
        
        if not train_flag:
            global val_idx
            val_idx = filtered_idx

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
            x = pack(torch.tensor(self.word_level[idx]), 
                                  torch.tensor(self.char_level[idx]))
            y = torch.tensor(self.labels[idx])
            return (x, y)

# Class disjoint training and validation set
train_dataset = EntriesDisjointDataset(df, True)
val_dataset = EntriesDisjointDataset(df, False)
assert set(train_dataset.labels).isdisjoint(set(val_dataset.labels))

In [18]:
def collate_fn(batch):
    '''Сollate_fn для корректного формирования даталоадеров 
    внутри trainer/tester из библиотеки PML. Формат выхода функции
    (x, y) синхронизирован с интерфейсом указанных классов'''
    word_level_arr = []
    char_level_arr = []
    label_arr = []
    for item in batch: 
        
        x, y = item
        word_level, char_level = unpack(x)
        word_level_arr.append(word_level)
        char_level_arr.append(char_level)
        label_arr.append(y)

    word_level_arr = pad_sequence(word_level_arr, batch_first=True)
    char_level_arr = pad_sequence(char_level_arr, batch_first=True)
    
    x = batch_pack(word_level_arr, char_level_arr)
    y = torch.tensor(label_arr)
    
    return (x, y)

# Конфигурация сети

LSTM - надежный базовый выбор для подобных задач; в частности, сведения последовательности переменной длины, где важен - хотя бы относительно - порядок, в вектор фиксированного размера. Блоки двунаправленные (поскольку сразу доступно все описание целиком), а также с двумя слоями для потенциального выделения более высокоуровневых признаков. Альтернативно можно было бы рассмотреть сверточную сеть с max pool over time (для выравнивания различий в длинах входящих последовательностей), а также трансформеры. Последние, как известно, очень удобно параллелизовать, что было бы преимуществом в контексте более широкой задачи.<br>
<br>
Из некоторых особенностей - мы замораживаем веса предобученных эмбеддингов, чтобы не произошло их дальнейшее переобучение вследствие небольшого размера выборки (в общем случае, для задач similarity рекомендации входные эмбеддинги дообучать). Для посимвольной ветви, разумеется, вариантов нет - используем обучаемый embedding слой.

In [19]:
#получаем веса эмбеддингов
embedding_weights = torch.FloatTensor(glove_vectors.vectors)

In [20]:
class MetricLSTM(nn.Module):
    def __init__(self):
        super(MetricLSTM, self).__init__()
     
        self.word_embedding = nn.Embedding.from_pretrained(embedding_weights, 
                                                      freeze=True,
                                                      padding_idx=0)
   
        self.word_lstm = nn.LSTM(input_size=25,
                            hidden_size=25,
                            num_layers=2,
                            batch_first=True, 
                            bidirectional=True)
        
        self.char_embedding = nn.Embedding(num_embeddings=n_chars, #было плохо
                                           embedding_dim=25, 
                                           padding_idx=0)
        
        self.char_lstm = nn.LSTM(input_size=25,
                                hidden_size=25,
                                num_layers=2,
                                batch_first=True, 
                                bidirectional=True)
        
        self.linear = nn.Linear(25*2*2*2, 100)
    
    def forward_branch(self, x, embedding, lstm):
        x = embedding(x) #(N, L, 25)
        x = lstm(x)[1][0] #(4, N, 25)
        x = x.permute(1, 0, -1) #(N, 4, 25)
        x = torch.cat((torch.chunk(x, 4, dim=1)[0], 
                       torch.chunk(x, 4, dim=1)[1],
                       torch.chunk(x, 4, dim=1)[2],
                       torch.chunk(x, 4, dim=1)[3]), dim=2)
                        #(N, 1, 100)
        x = x.squeeze(dim=1) #(N, 100)
        return x

    def forward(self, x):
        x_word, x_char = batch_unpack(x)
        x_word = self.forward_branch(x_word, self.word_embedding,
                                    self.word_lstm)
        x_char = self.forward_branch(x_char, self.char_embedding,
                                    self.char_lstm)
        x = torch.cat([x_word, x_char], dim=1)
        embedding = self.linear(x)
        
        return embedding

# Обучение

Процесс обучения адаптирован по следующим ноутбукам: <br>
https://github.com/KevinMusgrave/pytorch-metric-learning/blob/master/examples/notebooks/MetricLossOnly.ipynb
https://github.com/KevinMusgrave/pytorch-metric-learning/blob/master/examples/notebooks/scRNAseq_MetricEmbedding.ipynb <br>
<br>
Нашей искомой построчной f1 метрики нет в стандартных метриках PML, поэтому обучение контролируем по сильно коррелирующей с ней прокси-метрике (mean average precision at r из рекомендованной статьи Metric Learning: Reality Check). В идеале, было бы хорошо реализовать необходимую нам метрику как custom средствами библиотеки; в этом случае, помимо прочего, непосредственно по ней можно будет делать early stopping (параметр patience в end of epoch hook) и т.д.<br>
<br>
В качестве лосс-функции был выбран triplet loss (в сочетании с определенными функции подбора объектов при обучении), но здесь остается достаточно широкое пространство для экспериментирования и потенциального улучшения. Особо интересно выглядят методы с "искусственным центром" (Center Loss, SphereFace, ArcFace, CosFace), применяющиеся в SOTA моделях; их реализация есть в PML. <br> 
<br>
Некоторые затруднения вызвала некорректная локальная работа библиотеки faiss, на которую, в свою очередь, завязана часть PML; большинство затруднений удалось преодолеть через кастомизацию (например, исключив некоторые метрики из списка стандартно рассчитываемых, прописав отдельно функцию для KNN и тд), но в целом вопрос налаживания ее корректной работы очень важен.

### Инициализация модели и параметров

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trunk = MetricLSTM()
trunk_optimizer = torch.optim.Adam(trunk.parameters(), 
                                   lr=0.001,
                                   weight_decay=0.0001)

loss = losses.TripletMarginLoss(margin=0.1)

#Функция майнинга триплетов (определяет стратегию майнинга)
miner = miners.MultiSimilarityMiner(epsilon=0.1)

#Сэмплер
sampler = samplers.MPerClassSampler(train_dataset.labels, m=2, 
                                    length_before_new_iter=len(train_dataset))
#m=2, так как у нас может и не быть более чем двух примеров одного класса в целом батче

batch_size = 256
num_epochs = 50

#Запаковываем в словари
models = {"trunk": trunk}
optimizers = {"trunk_optimizer": trunk_optimizer}
loss_funcs = {"metric_loss": loss}
mining_funcs = {"tuple_miner": miner}

### Спец процедуры для вызова в процессе обучения

In [22]:
#главный контейнер процедур
record_keeper, _, _ = logging_presets.get_record_keeper("metriclstm_logs", 
                                                        "metriclstm_tensorboard")

hooks = logging_presets.get_hook_container(record_keeper,
                                            record_group_name_prefix=None, 
                                            primary_metric="mean_average_precision_at_r", 
                                            validation_split_name="val",
                                            save_models=True,
                                            log_freq=50) 

#тестировщик
knn_func = CustomKNN(SNRDistance())
accuracy_calculator = AccuracyCalculator(exclude=("NMI", "AMI"), 
                                        knn_func=knn_func)

tester = testers.GlobalEmbeddingSpaceTester(
                    end_of_testing_hook=hooks.end_of_testing_hook,
                    batch_size=128,
                    dataloader_num_workers=0,
                    accuracy_calculator=accuracy_calculator)

#процедура в конце эпохи
dataset_dict = {"val": val_dataset} 
model_folder = "metriclstm_saved_models"

end_of_epoch_hook = hooks.end_of_epoch_hook(tester, dataset_dict, model_folder, 
                                            test_interval=1, patience=5,
                                            test_collate_fn=collate_fn)

### Сборка и инициализация trainer'a

In [23]:
trainer = trainers.MetricLossOnly(
    models,
    optimizers,
    batch_size,
    loss_funcs,
    mining_funcs,
    train_dataset,
    sampler=sampler,
    collate_fn=collate_fn,
    dataloader_num_workers=0,
    end_of_iteration_hook=hooks.end_of_iteration_hook,
    end_of_epoch_hook=end_of_epoch_hook,
)

### Запуск визуализации в Тензорборд

In [34]:
%load_ext tensorboard
%tensorboard --logdir metriclstm_tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Обучение модели

Модель обучается в рабочем режиме; обучение заканчивается на 40-й эпохе, когда метрика выходит на плато при условии достаточно щадящего значения patience=5.

In [25]:
trainer.train(num_epochs=num_epochs)
#выводимая метрика: mean average precision at r

INFO:PML:Initializing dataloader
INFO:PML:Initializing dataloader iterator
INFO:PML:Done creating dataloader iterator
INFO:PML:TRAINING EPOCH 1
total_loss=0.11147: 100%|███████████████████████| 41/41 [00:15<00:00,  2.72it/s]
INFO:PML:Evaluating epoch 1
INFO:PML:Getting embeddings for the val split
100%|███████████████████████████████████████████| 33/33 [00:02<00:00, 15.22it/s]
INFO:PML:Computing accuracy for the val split w.r.t ['val']
INFO:PML:New best accuracy! 0.3506856742770156
INFO:PML:TRAINING EPOCH 2
total_loss=0.12309: 100%|███████████████████████| 41/41 [00:15<00:00,  2.71it/s]
INFO:PML:Evaluating epoch 2
INFO:PML:Getting embeddings for the val split
100%|███████████████████████████████████████████| 33/33 [00:02<00:00, 14.71it/s]
INFO:PML:Computing accuracy for the val split w.r.t ['val']
INFO:PML:New best accuracy! 0.3900538907389589
INFO:PML:TRAINING EPOCH 3
total_loss=0.10559: 100%|███████████████████████| 41/41 [00:16<00:00,  2.50it/s]
INFO:PML:Evaluating epoch 3
INFO:PML:

INFO:PML:TRAINING EPOCH 23
total_loss=0.10987: 100%|███████████████████████| 41/41 [00:14<00:00,  2.77it/s]
INFO:PML:Evaluating epoch 23
INFO:PML:Getting embeddings for the val split
100%|███████████████████████████████████████████| 33/33 [00:02<00:00, 14.99it/s]
INFO:PML:Computing accuracy for the val split w.r.t ['val']
INFO:PML:TRAINING EPOCH 24
total_loss=0.08965: 100%|███████████████████████| 41/41 [00:14<00:00,  2.85it/s]
INFO:PML:Evaluating epoch 24
INFO:PML:Getting embeddings for the val split
100%|███████████████████████████████████████████| 33/33 [00:02<00:00, 15.11it/s]
INFO:PML:Computing accuracy for the val split w.r.t ['val']
INFO:PML:New best accuracy! 0.540953991437213
INFO:PML:TRAINING EPOCH 25
total_loss=0.06549: 100%|███████████████████████| 41/41 [00:15<00:00,  2.73it/s]
INFO:PML:Evaluating epoch 25
INFO:PML:Getting embeddings for the val split
100%|███████████████████████████████████████████| 33/33 [00:02<00:00, 15.07it/s]
INFO:PML:Computing accuracy for the val sp

# Инференс и подсчет требуемой метрики

### Загрузка модели (при необходимости)

In [27]:
trunk = MetricLSTM()
trunk.load_state_dict(torch.load('metric_lstm.pth'))

<All keys matched successfully>

In [26]:
trunk.eval()

MetricLSTM(
  (word_embedding): Embedding(1193514, 25, padding_idx=0)
  (word_lstm): LSTM(25, 25, num_layers=2, batch_first=True, bidirectional=True)
  (char_embedding): Embedding(69, 25, padding_idx=0)
  (char_lstm): LSTM(25, 25, num_layers=2, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=200, out_features=100, bias=True)
)

### Возвращаем валидационный датафрейм и считаем эмбеддинги на его объектах

In [27]:
#индексы сохранены предварительно на этапе создания датасетов
val_df = df.iloc[val_idx, :].reset_index(drop=True)
val_df

,posting_id,word_level,char_level,label_group,matches
0,train_2406599165,"[131370, 18071, 46352, 11005, 28, 32674, 37431...","[16, 16, 0, 39, 50, 46, 39, 42, 47, 0, 42, 54,...",6300,train_2406599165 train_3342059966
1,train_3342059966,"[131370, 168647, 102671, 11235, 17, 792, 423, ...",[0],6300,train_2406599165 train_3342059966
2,train_998568945,"[8227, 6734, 289536, 40421, 3998, 5356, 76204,...","[21, 16, 21, 16, 0, 58, 21]",6475,train_998568945 train_3118756415
3,train_3118756415,"[8227, 6734, 6734, 39601, 79764, 1494, 40421, ...","[17, 57, 51, 42, 0, 21, 16, 21, 16, 0, 42, 41,...",6475,train_998568945 train_3118756415
4,train_1461445798,"[18509, 10796, 11520, 35, 38, 490, 2274, 521, ...","[17, 0, 17, 0, 21, 0, 21]",6559,train_1461445798 train_1749322479
...,...,...,...,...,...
4198,train_3637239268,"[202997, 150136]","[20, 16, 16, 45, 56, 0, 39, 52, 51, 59, 51]",5824,train_1643107217 train_3637239268
4199,train_3391985594,"[449, 952, 37259, 39652, 39652, 46352, 11005, ...","[17, 18, 18, 21, 0, 17, 18, 18, 22, 0, 17, 18,...",5856,train_3391985594 train_1712739855
4200,train_1712739855,"[37259, 39652, 28, 39652, 5555, 1568, 1617, 44...","[17, 18, 18, 21, 0, 49, 43, 51, 43, 48, 39, 50...",5856,train_3391985594 train_1712739855
4201,train_2944131255,"[105361, 4088, 3039, 3190, 38, 38, 38, 72, 137...","[19, 16, 0, 39, 18, 16, 0, 39, 19, 16, 0, 39, ...",5552,train_2944131255 train_1001474240


In [28]:
def to_inference_tensor(array):
    '''Служебная функция для перевода информации из датафрейма
    в формат, требуемый на вход в MetricLSTM'''
    tensor1 = torch.tensor(array[0]).unsqueeze(dim=0)
    tensor2 = torch.tensor(array[1]).unsqueeze(dim=0)
    inference_tensor = batch_pack(tensor1, tensor2)
    return inference_tensor

In [29]:
val_df['embedding'] = val_df[['word_level', 'char_level']].values.tolist()
val_df['embedding'] = val_df['embedding'].apply(to_inference_tensor)
val_df['embedding'] = val_df['embedding'].apply(lambda x: 
                                                trunk(x).detach().numpy().tolist()[0])
val_df

,posting_id,word_level,char_level,label_group,matches,embedding
0,train_2406599165,"[131370, 18071, 46352, 11005, 28, 32674, 37431...","[16, 16, 0, 39, 50, 46, 39, 42, 47, 0, 42, 54,...",6300,train_2406599165 train_3342059966,"[0.23456330597400665, -0.20327991247177124, -0..."
1,train_3342059966,"[131370, 168647, 102671, 11235, 17, 792, 423, ...",[0],6300,train_2406599165 train_3342059966,"[0.25036174058914185, -0.06551595777273178, 0...."
2,train_998568945,"[8227, 6734, 289536, 40421, 3998, 5356, 76204,...","[21, 16, 21, 16, 0, 58, 21]",6475,train_998568945 train_3118756415,"[0.06010384485125542, 0.00598347932100296, -0...."
3,train_3118756415,"[8227, 6734, 6734, 39601, 79764, 1494, 40421, ...","[17, 57, 51, 42, 0, 21, 16, 21, 16, 0, 42, 41,...",6475,train_998568945 train_3118756415,"[-0.13363735377788544, 0.1606520414352417, 0.0..."
4,train_1461445798,"[18509, 10796, 11520, 35, 38, 490, 2274, 521, ...","[17, 0, 17, 0, 21, 0, 21]",6559,train_1461445798 train_1749322479,"[0.10545273125171661, -0.0020801499485969543, ..."
...,...,...,...,...,...,...
4198,train_3637239268,"[202997, 150136]","[20, 16, 16, 45, 56, 0, 39, 52, 51, 59, 51]",5824,train_1643107217 train_3637239268,"[0.19476942718029022, -0.07286263257265091, -0..."
4199,train_3391985594,"[449, 952, 37259, 39652, 39652, 46352, 11005, ...","[17, 18, 18, 21, 0, 17, 18, 18, 22, 0, 17, 18,...",5856,train_3391985594 train_1712739855,"[0.34747713804244995, -0.12604424357414246, -0..."
4200,train_1712739855,"[37259, 39652, 28, 39652, 5555, 1568, 1617, 44...","[17, 18, 18, 21, 0, 49, 43, 51, 43, 48, 39, 50...",5856,train_3391985594 train_1712739855,"[0.250297486782074, -0.11767282336950302, -0.1..."
4201,train_2944131255,"[105361, 4088, 3039, 3190, 38, 38, 38, 72, 137...","[19, 16, 0, 39, 18, 16, 0, 39, 19, 16, 0, 39, ...",5552,train_2944131255 train_1001474240,"[0.20644812285900116, -0.08114614337682724, 0...."


### Поиск объектов, близких по мере, и подсчет итоговой метрики

Сравнивая эмбеддинги по мере (в данном случае по косинусной близости) с определенным порогом, мы поочередно ищем объекты, с высокой вероятностью одинаковые с данным. Диапазон порога, который обычно используется, оценил бы как [0.7, 0.9]; остановился на значении 0.8. <br>
<br>
Необходимо сразу отметить, что реализованный в настоящий момент алгоритм вычисления схожих объектов квадратичный и нуждается в оптимизации. Ей будут способствовать решение локальной проблемы с faiss, а также адаптация под задачу соответствующих классов PML

In [30]:
def f1_score_lists(y_true, y_pred):
    '''slightly different input format relative to the original
    input: two lists, output: float'''
    y_true = [set(x.split()) for x in y_true]
    y_pred = [set(x) for x in y_pred]
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)]) 
    len_y_true = np.array([len(x) for x in y_true])
    len_y_pred = np.array([len(x) for x in y_pred])
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output

threshold = 0.80
n = len(val_df)
predicted_matches = [None]*len(val_df)

for i in val_df.index[:n]:
    clear_output(wait=True)
    print(i, ' / ', n-1)
    current_predicted_match = []
    for j in val_df.index:
        vec1 = np.array([val_df.iloc[i]['embedding']])
        vec2 = np.array([val_df.iloc[j]['embedding']])
        if cosine_similarity(vec1, vec2) >= threshold:
            current_predicted_match.append(val_df.iloc[j]['posting_id'])
    predicted_matches[i] = current_predicted_match

4202  /  4202


### Финальная оценка

Удалось добиться существенного улучшения на валидационной выборке по сравнению с бейзлайном (0.60 против 0.36).

In [33]:
y_true = list(val_df['matches'][:n])
y_pred = predicted_matches[:n]

score = f1_score_lists(y_true, y_pred).mean()
print('F1 score: ', f'{score:.2f}')

F1 score:  0.60


## Ссылки и справочные материалы

Материалы (из описания задачи):
- https://github.com/scikit-learn-contrib/metric-learn
- https://arxiv.org/pdf/1503.03832.pdf
- https://github.com/KevinMusgrave/pytorch-metric-learning
- https://arxiv.org/pdf/2003.08505.pdf

Дополнительные материалы, которые оказались полезными: <br>
- Ха Ву Тран (Microsoft), лекция по Metric Learning <br>
https://www.youtube.com/watch?v=aU9yEwgrJ54 <br>
- Benyamin Ghojogh, Ali Ghodsi, Fakhri Karray, Mark Crowley. Spectral, Probabilistic, and Deep Metric Learning: Tutorial and Survey <br>
https://arxiv.org/abs/2201.09267